<a href="https://colab.research.google.com/github/DatNguyen2084/DLDH-Metaphor-detection/blob/main/MBERT_Metaphor_classification_Intermediate_task_TroFi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intermediate-task fine-tunning MBERT for Metaphor detection
We apply here the principle of transfer learning. We train the based MBERT on the intermediate-task with the TroFi-dataset. The trained model will be used for the final task on KontextBruch-dataset

## Install transformers and import packages

In [ ]:
!pip install -q sentence_transformers
!pip install -q datasets
!pip install seqeval

     |████████████████████████████████| 78 kB 3.6 MB/s 
     |████████████████████████████████| 3.5 MB 12.0 MB/s 
     |████████████████████████████████| 6.8 MB 30.1 MB/s 
     |████████████████████████████████| 1.2 MB 41.7 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 44.7 MB/s 
     |████████████████████████████████| 895 kB 42.7 MB/s 
     |████████████████████████████████| 311 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 29.6 MB/s 
     |████████████████████████████████| 133 kB 33.5 MB/s 
     |████████████████████████████████| 243 kB 44.0 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 271 kB 44.1 MB/s 
     |████████████████████████████████| 144 kB 34.6 MB/s 
     |████████████████████████████████| 43 kB 1.1 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=9d3b8d7268f52e595964ea6fe204cad29f4cda5c11dc5c92e804

In [ ]:
import pandas as pd
import os
import os.path
import numpy as np
import torch
from sklearn.model_selection import train_test_split

##Utils

### Do we have GPU?

In [ ]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

### Mount to data folder

In [ ]:
# Mount Google Drive
# The following data is needed: https://drive.google.com/drive/folders/1uPnLexQh8kbV5ErVR7ksagVKP_wDd4a0?usp=sharing
# Create a shortcut to your Drive ("Drive-Verknüpfung hinzufügen" zu "Meine Ablage")
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT_PATH = '/content/drive/My Drive/DLDH'
DATA_PATH = '/data'
MODEL_PATH = '/model'
MIP_PATH = '/content/drive/My Drive/Annotationen - MIP - 11 Datensätze'

### Processing TroFi-dataset

In [ ]:
trofi_df = pd.read_csv(ROOT_PATH + DATA_PATH + '/Trofi/TroFiExampleBase.txt', sep='\t', skiprows=2, header=None, names=['id', 'label', 'text'])
# drop row where label is NaN
trofi_df = trofi_df.dropna()
# drop row where label is U (Unnotated)
trofi_df = trofi_df.drop(trofi_df[(trofi_df['label'] == 'U')].index)
trofi_df.loc[trofi_df['label'] == 'N', 'label'] = 1
trofi_df.loc[trofi_df['label'] == 'L', 'label'] = 0
print("Label 0")
print(len(trofi_df[(trofi_df['label'] == 0)]))
print("label 1")
print(len(trofi_df[(trofi_df['label'] == 1)]))

Label 0
1592
label 1
2145


## TroFi Intermediate-task Fine-tunning

### Define Dataset

In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score
  
class MetaphorDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

###Compute Metric
we compute the marco-F1 score for the evaluation of training epochs

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    macro_f1 = f1_score(labels, preds, average='macro')
    return {
        'macro_f1': macro_f1,
    }

### Load MBERT and Tokenizer

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification, AutoTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
model = BertForSequenceClassification.from_pretrained(
        "bert-base-multilingual-cased",
        cache_dir=None,
        num_labels=2,
        #id2label=id2tag,
        #label2id=tag2id,
    )
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

### Training

In [ ]:
max_length = 512
epochs = 20
train_df, testdf = train_test_split(trofi_df, test_size=0.2, random_state=32)

train_encodings = tokenizer(train_df['text'].to_list(), padding=True, truncation=True, max_length=max_length)
test_encodings = tokenizer(testdf['text'].to_list(), padding=True, truncation=True, max_length=max_length)

train_dataset = MetaphorDataset(train_encodings, train_df['label'].values)
val_dataset = MetaphorDataset(test_encodings, testdf['label'].values)


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=epochs,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    save_total_limit=1,
    save_strategy="epoch",
    logging_strategy="epoch",
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    metric_for_best_model="macro_f1",
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

# Save model and tokenizer after training
trainer.save_model(ROOT_PATH + "/intermediate-task-trofi/model")
tokenizer.save_pretrained(ROOT_PATH + "/intermediate-task-trofi/model")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2989
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7480


Epoch,Training Loss,Validation Loss,Macro F1
1,0.552400,0.542677,0.799414
2,0.461000,0.642786,0.759971
3,0.360900,0.615705,0.800645
4,0.291400,0.836320,0.754667
5,0.180100,0.833640,0.791488
6,0.129300,0.819231,0.821165
7,0.068300,1.064728,0.812927
8,0.046500,1.312904,0.795733
9,0.040800,1.224098,0.805259


***** Running Evaluation *****
  Num examples = 748
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-374
Configuration saved in ./results/checkpoint-374/config.json
Model weights saved in ./results/checkpoint-374/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 748
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-748
Configuration saved in ./results/checkpoint-748/config.json
Model weights saved in ./results/checkpoint-748/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 748
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1122
Configuration saved in ./results/checkpoint-1122/config.json
Model weights saved in ./results/checkpoint-1122/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-374] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 748
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-1496
Configuration saved in ./results/checkpoint-1496/c

('/content/drive/My Drive/DLDH/intermediate-task-trofi/model/tokenizer_config.json',
 '/content/drive/My Drive/DLDH/intermediate-task-trofi/model/special_tokens_map.json',
 '/content/drive/My Drive/DLDH/intermediate-task-trofi/model/vocab.txt',
 '/content/drive/My Drive/DLDH/intermediate-task-trofi/model/added_tokens.json',
 '/content/drive/My Drive/DLDH/intermediate-task-trofi/model/tokenizer.json')